Build Summary Tables  
    
      
      
      
1. Sales by date by hour:

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# Define project and dataset IDs as variables
project_id = 'umt-msba'
dataset_id = 'transactions'

# Query to generate the 'Sales by date by hour' summary table
query_sales_by_hour = f"""
    SELECT
        DATE(datetime) AS calendar_date,
        EXTRACT(HOUR FROM datetime) AS hour,
        SUM(total) AS total_spend,
        COUNT(datetime) AS num_transactions,
        SUM(CASE WHEN trans_status = ' ' OR trans_status = '' THEN 1 ELSE 0 END) AS num_items
    FROM `{project_id}.{dataset_id}.transArchive_*`
    WHERE CAST(card_no AS INT64) != 3  -- Exclude non-owners
    GROUP BY calendar_date, hour
    ORDER BY calendar_date, hour
"""

# Execute the query and fetch the data
df_sales_by_hour = client.query(query_sales_by_hour).to_dataframe()

# Display the result
print(df_sales_by_hour.head())





2. Sales by owner by year by month:

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# Define project and dataset IDs as variables
project_id = 'umt-msba'
dataset_id = 'transactions'

# Query to generate the 'Sales by owner by year by month' summary table
query_sales_by_owner = f"""
    SELECT
        CAST(card_no AS INT64) AS card_no,  -- Owner ID
        EXTRACT(YEAR FROM datetime) AS year,
        EXTRACT(MONTH FROM datetime) AS month,
        SUM(total) AS total_sales,
        COUNT(datetime) AS num_transactions,
        SUM(CASE WHEN trans_status = ' ' OR trans_status = '' THEN 1 ELSE 0 END) AS num_items
    FROM `{project_id}.{dataset_id}.transArchive_*`
    WHERE CAST(card_no AS INT64) != 3  -- Exclude non-owners
    GROUP BY card_no, year, month
    ORDER BY card_no, year, month
"""

# Execute the query and fetch the data
df_sales_by_owner = client.query(query_sales_by_owner).to_dataframe()

# Display the result
print(df_sales_by_owner.head())




3. Sales by product description by year by month:

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client()

# Define project and dataset IDs as variables
project_id = 'umt-msba'
dataset_id = 'transactions'

# Query for 'Sales by product description by year by month'
query_sales_by_product = f"""
    WITH sales_data AS (
        SELECT
            upc,
            description,
            department,
            EXTRACT(YEAR FROM datetime) AS year,
            EXTRACT(MONTH FROM datetime) AS month,
            SUM(total) AS total_sales,
            COUNT(datetime) AS num_transactions,
            SUM(CASE WHEN trans_status = ' ' OR trans_status = '' THEN 1 ELSE 0 END) AS num_items
        FROM `{project_id}.{dataset_id}.transArchive_*`
        WHERE CAST(card_no AS INT64) != 3  -- Exclude non-owners
        GROUP BY upc, description, department, year, month
    )
    SELECT sd.*, dl.dept_name
    FROM sales_data sd
    LEFT JOIN `{project_id}.{dataset_id}.department_lookup` dl
    ON sd.department = dl.department
    ORDER BY year, month
"""

# Execute the query and fetch the data
df_sales_by_product = client.query(query_sales_by_product).to_dataframe()

# Display the result
print(df_sales_by_product.head())


Create SQLite Database and Insert Data

In [ ]:
import sqlite3  
import pandas as pd

# Create SQLite connection and cursor
conn = sqlite3.connect('wedge_coop_summary.db')
cursor = conn.cursor()

# Drop the tables if they already exist
cursor.execute('DROP TABLE IF EXISTS sales_by_hour')
cursor.execute('DROP TABLE IF EXISTS sales_by_owner')
cursor.execute('DROP TABLE IF EXISTS sales_by_product')

# Create the tables in SQLite
cursor.execute('''
CREATE TABLE sales_by_hour (
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER,
    total_spend REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_owner (
    card_no INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_product (
    upc TEXT,
    description TEXT,
    department INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

# Insert data into SQLite tables
# Insert sales by hour
df_sales_by_hour.to_sql('sales_by_hour', conn, if_exists='replace', index=False)

# Insert sales by owner
df_sales_by_owner.to_sql('sales_by_owner', conn, if_exists='replace', index=False)

# Insert sales by product
df_sales_by_product.to_sql('sales_by_product', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Summary tables created and stored in 'wedge_coop_summary.db'")
